In [38]:
import praw
import pandas as pd
from praw.models import MoreComments
import re

In [51]:
# Read-only instance
reddit_read_only = praw.Reddit(client_id="K85pc4mBX5iYnRsUF-iEdQ",		 # your client id
							client_secret="cDSwL4YAtDTkQsX9Risd5wnBafORGw",	 # your client secret
							user_agent="idk")	 # your user agent

subreddit = reddit_read_only.subreddit("berkeley")
#posts = subreddit.top(limit=20)

In [52]:
posts = subreddit.top(time_filter="year", limit=None)
# Scraping the top posts of the current year
 
posts_dict = {"Title": [], "Post Text": [], "Score": [],
              "Comments": [], "EECS": []
              }

In [53]:
for post in posts:
    # Title of each post
    posts_dict["Title"].append(post.title)
     
    # Text inside a post
    posts_dict["Post Text"].append(post.selftext)
     
    # The score of a post
    posts_dict["Score"].append(post.score)
     
    comments = []
    for top_level_comment in post.comments:
        if isinstance(top_level_comment, MoreComments):
            continue
        elif (top_level_comment.body != "[deleted]"):
            comments.append(top_level_comment.body)
        else:
            comments.append('')
    # Total number of comments inside the post
    posts_dict["Comments"].append(comments)
     

	# Flair
    if post.link_flair_text=="CS/EECS" or bool(re.search(r"\b[Cc][Ss]\s?\d+[ABCabc]?", post.title + " " + post.selftext)):
        posts_dict["EECS"].append(1)
    else:
	    posts_dict["EECS"].append(0)
posts_dict

{'Title': ['URGENT: MISSING STUDENT. PLEASE LET US KNOW IF YOU HAVE SEEN HIM.',
  "standing ovation for Professor Hilfinger's last lecture after 40 years of teaching",
  'Saw Christopher Nolan on campus today possibly scouting for his new film Oppenheimer.',
  'CAL WINS 124TH BIG GAME 41-11',
  'Had some free time before the semester started, so I made a Berkeley slander video',
  'aerial view of the glade on 4/20/22 @ 4:20PM',
  'ok now that’s just silly',
  "Berkeley book fair today. Can I convince anyone to go buy my dad's book? I'll reimburse you for it.",
  'I built the Campanile out of legos!',
  'to all the non bay area ppl talking about mexican food',
  'it’s not thaaaat bad',
  'Update: reviewing all the crispest water on campus',
  'NIMBYs watch in horror as college town builds more housing for students',
  'Lowest Score in CS 170',
  'Intersections of Berkeley',
  'I think my GSI is flirting with me...',
  'Things are really hard for current bears. Who needs a Costco trip.',

In [54]:
# Saving the data in a pandas dataframe
top_posts = pd.DataFrame(posts_dict)
eecs = top_posts.loc[top_posts['EECS'] == 1]
no = top_posts.loc[top_posts['EECS'] == 0]
top_posts


Title  \
0    URGENT: MISSING STUDENT. PLEASE LET US KNOW IF...   
1    standing ovation for Professor Hilfinger's las...   
2    Saw Christopher Nolan on campus today possibly...   
3                        CAL WINS 124TH BIG GAME 41-11   
4    Had some free time before the semester started...   
..                                                 ...   
995  whoever made this to get in a party actually h...   
996  Happy first day of classes! If your landlord r...   
997                        school’s grad ticket scam??   
998         UC Berkeley Class Action Lawsuit: CS Fraud   
999  UC Berkeley will increase California students,...   

                                             Post Text  Score  \
0    Hello, our friend Luca Elghanayan has been mis...   2496   
1                                                        1362   
2                                                        1225   
3                                                         951   
4                                                         938   
..                                                 ...    ...   
995                                                       131   
996                                                       130   
997  So I, like all my other graduating friends, go...    128   
998  **It's completely fine if we don't major in CS...    131   
999                                                       130   

                                              Comments  EECS  
0    [He is gone. He was my friend. Rest in peace.,...     0  
1    [Shame I had office hours for students; anyway...     1  
2    [this is so sick. gotta casually stroll around...     0  
3    [Garbers undefeated at Stanford Stadium, GO BE...     0  
4    [Lol, I wish I had an award to give this, have...     0  
..                                                 ...   ...  
995  [who cares this much 😭 is it any surprise that...     0  
996  [>TFW you're a ticket scalper, but for living ...     0  
997  [also the worst graduation ever planned, When ...     0  
998  [, Isn’t this all speculation? Am I missing so...     1  
999  [Supporting this is fine but better not compla...     0  

[1000 rows x 5 columns]

In [55]:
import cohere
from cohere.classify import Example

In [56]:
eecs["Comments"].tolist()


[['Shame I had office hours for students; anyway, happy retirement, and all my best wishes for Prof. Hilfinger!',
  'Best wishes for professor during his retirement! Thank you for teaching the class which made me love coding!',
  'Hopefully the modern era of academia means that future greats of teaching have their lectures recorded for all those in the future to reference.\n\nCongratulations to Hilfinger for a long and illustrious career, from myself and all the other students whose life he has enriched!',
  'What were his last words? "RTFM"?',
  'Is there a recording of it anywhere?',
  'Gitlet\nOf\nAll\nTime',
  'Former CS164 TA here. Loved working with Prof. Hilfinger. It was a very rewarding experience. I’m sure his students feel the same.',
  'My dad and I both had him during undergrad, 36 years apart. Truly the end of an era. Happy retirement to him!',
  'He finally retired?  Good for him!\n\nIs there any prof to take his place as "holy shit you\'ll learn a lot if you take it fro

In [62]:
from praw.reddit import re


#apiKey = 'a5QDa98c2KhjCtdXkrC5yyxcCPxcnQneFgIuycIJ'
apiKey = '5HtwM7zol4KLjnUtb6I49s98eSfTpXMQ0f5cXHdr'
co = cohere.Client(apiKey)
bleh = []
for i in range(len(eecs['Comments'].tolist())):
    print("Debug: ")
    print(i)
    inpts = [s for s in eecs['Comments'].tolist()[i] if s != '']
    response = co.classify(model='cohere-toxicity', inputs=inpts)
    num_tox = 0;
    for i in range(len(inpts)): 
        if response.classifications[i].__getattribute__('prediction') == 'TOXIC':
            num_tox += 1
    if num_tox/len(inpts) >= 0.85:
        bleh.append('LEAGUE TOXIC')
    elif num_tox/len(inpts) >= 0.5:
        bleh.append('TOXIC')
    else:
        bleh.append('NOT TOXIC')

print('The labels are: {}'.format(bleh))

TypeError: can only concatenate str (not "int") to str

In [47]:
bleh2 = []
for i in range(len(no['Comments'].tolist())):
    inpts2 = [s for s in no['Comments'].tolist()[i] if s != '']
    response = co.classify(model='cohere-toxicity', inputs=inpts2)
    num_tox = 0;
    for i in range(len(inpts2)): 
        if response.classifications[i].__getattribute__('prediction') == 'TOXIC':
            num_tox += 1
    if num_tox/len(inpts2) >= 0.85:
        bleh2.append('LEAGUE TOXIC')
    elif num_tox/len(inpts2) >= 0.5:
        bleh2.append('TOXIC')
    else:
        bleh2.append('NOT TOXIC')

In [50]:
print("Percent of Non-CS/EECS posts that are classified as Toxic: ")
print(sum([bleh2[i] == "TOXIC" for i in range(len(bleh2))]) / len(bleh2))
print('\n')
print("Percent of CS/EECS posts that are classified as Toxic: ")
print(sum([bleh[i] == "TOXIC" for i in range(len(bleh))]) / len(bleh))
print("Percent of Total Posts Classified as Toxic: ")
print((sum([bleh2[i] == "TOXIC" for i in range(len(bleh2))]) + sum([bleh[i] == "TOXIC" for i in range(len(bleh))]))/(len(bleh2)+len(bleh)))

Number of Non-CS/EECS posts that are classified as Toxic: 
0.12359550561797752


Number of CS/EECS posts that are classified as Toxic: 
0.0
